In [1]:
#jonathan Lord


In [1]:
#libraries
from bs4 import BeautifulSoup
import requests
import openpyxl
import re
from openpyxl import load_workbook
import collections
#import grequests



In [2]:
titles=["Rt", "Right", "Hon", "Sir", "Mr", "Mrs", "Ms", "Dr", "The", "the", "Rt", "Hon.", "Sir", "Lord", "Viscount", "Earl", "Archbishop","Baroness", "Lady", "Most", "Rev.", "Countess", "Marquess", "Duke", "Bishop"]
suffixes=["MP", "QC","OBE", "GCMG", "CBE", "KBE", "JP", "CH", "DL", "MBE", "CMG", "GCVO", "KCVO" "DCVO",  "CVO", "LVO","KT","KBE","OM", "MVO","GCB", "QSO", "KCMG", "DBE", "KT", "MRICS", "FREng", "FRS", "GBE", "KCVO", "OJ", "KG", "PC", "FSA","Kt","QPM","KCB","GBE","FRS","FBA","DSC","GCMG","LG","QPM","JP","CB"]

In [3]:
activeMP = []
url = "https://www.parliament.uk/mps-lords-and-offices/mps/find-your-mp/"
page = requests.get(url).content
soup = BeautifulSoup(page, 'html.parser')
MP_link_list = soup.find_all("a", href = re.compile('biographies/commons'))

for MP_link in MP_link_list:
    number = MP_link["href"].split("/")[6]
    
    contact = "https://members.parliament.uk/member/{}/contact".format(number)
    contact_page = requests.get(contact).content
    contact_soup = BeautifulSoup(contact_page, 'html.parser')
    
    career = "https://members.parliament.uk/member/{}/career".format(number)
    career_page = requests.get(career).content
    career_soup = BeautifulSoup(career_page, 'html.parser')
    
    activeMP.append([career_soup,contact_soup])

print(len(activeMP))

    

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
lords_pages = []
activeLord = []
for i in range(1,40):
    url = "https://members.parliament.uk/members/Lords?page={}".format(i)
    page = requests.get(url).content
    soup = BeautifulSoup(page, 'html.parser')
    lords_pages.append(soup)
    
for lords_page in lords_pages:
    lord_link_list = lords_page.find_all("a", href = re.compile('/contact'))
    
    for lord_link in lord_link_list:
        number = lord_link["href"].split("/")[2]
        
        contact = "https://members.parliament.uk/member/{}/contact".format(number)
        #print(contact)
        contact_page = requests.get(contact).content
        contact_soup = BeautifulSoup(contact_page, 'html.parser')
    
        career = "https://members.parliament.uk/member/{}/career".format(number)
        career_page = requests.get(career).content
        career_soup = BeautifulSoup(career_page, 'html.parser')
        
        activeLord.append([career_soup,contact_soup])
print(len(activeLord))
#print(activeLord[0])

In [ ]:
wb = load_workbook('Test.xlsx')
MPsh=wb["MPs"]
Lordsh=wb["Lords"]

In [ ]:
#MPcareer&Contact

for MP in activeMP:

    current=MP[0].find("div", {"card-box"})
    h4categories=current.find_all("h4")
    h4catstrips=[h4.string.strip() for h4 in h4categories]
    divcategories=current.find_all("div", {"card-list"})
    informationdict={}
    for h4,div in zip(h4catstrips,divcategories):
        if "Representation" in h4:
            info=div.find("div", {"class":"primary-info"}).string.strip()
        elif "Party affiliation" in h4:
            info=div.find("div", {"class":"primary-info"}).string.strip()
        else:
            if "Committee" in h4:
                infon=div.find_all("div", {"class":"primary-info"})
                info=[]
                for info1 in infon:
                    a=info1.string.strip()
                    info.append(a)
            elif "Other Post" in h4:
                infon=div.find_all("div", {"class":"primary-info"})
                info=[]
                for info1 in infon:
                    a=info1.string.strip()
                    info.append(a)
            else:
                #print(h4,div)
                info=[]
                positions=[]
                departments=[]
                info1=div.find_all("div", {"class":"primary-info"})
                for info1posi in info1:
                    a=info1posi.string.strip()
                    positions.append(a)
                    #print(info)
                try:
                    info2=div.find_all("div", {"class":"secondary-info"})
                
                    for info2posi in info2:
                        b=info2posi.string.strip()
                        departments.append(b)
                        positionanddepartment=[a,b]
                    
                except:
                    info2=None
                
                positionanddepartment=[a,b]
                for posi,dep in zip(positions,departments):
                    info.append([posi,dep])
          
        informationdict[h4]=info
        
#MP contact
    fullname=MP[1].find("h1").string.strip()
    #print(fullname)
    fir=[]
    sec=[]
    tit=[]
    suf=[]
    namesplit=[]
    splitnam=fullname.split(" ")
    for snam in splitnam:
        if snam in titles:
            tit.append(snam)
        elif snam in suffixes:
            suf.append(snam)
        else:
            namesplit.append(snam)
    for index, namez in enumerate(namesplit):
        if index==0:
            fir.append(namez)
        else:
            sec.append(namez)
    
    contmethods=MP[1].find_all("div",{"class":"card-inner"})
    for contmethod in contmethods:
        method=contmethod.find("div", {"class":"primary-info"}).string.strip()
        if "Parliamentary" in method:
            detail={}
            tradconts=contmethod.find_all("div", {"class":"contact-line"})
            addressall=contmethod.find("div", {"class":"col-md-5"}).get_text("\n").strip()
            addresssplit=addressall.splitlines()
            
            for tradcont in tradconts:
                label=tradcont.find("span", {"class":"label"}).string
                inform=tradcont.find("a").string.strip()
                detail[label]=inform
            detail["Address"]=addresssplit
        
        elif "Constituency" in method:
            detail={}
            tradconts=contmethod.find_all("div", {"class":"contact-line"})
            addressall=contmethod.find("div", {"class":"col-md-5"}).get_text("\n").strip()
        
            addresssplit=addressall.splitlines()
            
            #print(full1)
            for tradcont in tradconts:
                label=tradcont.find("span", {"class":"label"}).string.strip()
                inform=tradcont.find("a").string.strip()
                detail[label]=inform
            detail["Address"]=addresssplit
     ######################################################   
        elif "External/Private" in method:
            detail={}
            tradconts=contmethod.find_all("div", {"class":"contact-line"})
            addressall=contmethod.find("div", {"class":"col-md-5"}).get_text("\n").strip()
            addresssplit=addressall.splitlines()
            
            for tradcont in tradconts:
                label=tradcont.find("span", {"class":"label"}).string.strip()
                inform=tradcont.find("a").string.strip()
                detail[label]=inform
            detail["Address"]=addresssplit
            
        elif "Departmental" in method:
            detail={}
            tradconts=contmethod.find_all("div", {"class":"contact-line"})
            try:
                addressall=contmethod.find("div", {"class":"col-md-5"}).get_text("\n").strip()
                addresssplit=addressall.splitlines()
            except:
                addressall=None
            for tradcont in tradconts:
                label=tradcont.find("span", {"class":"label"}).string.strip()
                inform=tradcont.find("a").string.strip()
                detail[label]=inform
            detail["Address"]=addresssplit
        
        else:
            try:
                detail=contmethod.find("div", {"class":"indicator indicator-label"}).string.strip()
            except:
                detail=None
        methanddet=[method,detail]
        informationdict[method]=detail
        
    #career page variables  
    Constituency=informationdict.get("Representation")
    Party=informationdict.get("Party affiliation")
    OtherPost=informationdict.get("Other posts")
    if OtherPost!=None:
        Otherpost1=", ".join(filter(None,OtherPost))
    else:
        Otherpost1=None
    Committees=informationdict.get("Committee memberships")
    if Committees!=None:
        com_count=collections.Counter(Committees)
        chairlist=[]
        for key, value in com_count.items():
            if value > 1:
                chairlist.append(key)
        if chairlist!=None:
            for chaircom in chairlist:
                x=int(Committees.index(chaircom))
                Committees.insert(x, "CHAIR")
                Committees.remove(chaircom)
        Committees1=", ".join(Committees)
    else:
        Committees1=None
    posdep=[informationdict.get("Government post"),informationdict.get("Government posts"),\
informationdict.get("Opposition post"), informationdict.get("Opposition posts")]

    for postdescription in posdep:
        print(postdescription)
        if postdescription==None:
            continue
        
        else:
            thing=[]
            Position=postdescription[0][0]
            Department=postdescription[0][1]
            postdescription.pop(0)
            for a in postdescription:
                if a==None:
                    continue
                x=", ".join(a)
                thing.append(x)
            Additional_Posts="/".join(thing)
    if all(postdescription is None for postdescription in posdep):
            print("here")
            Position=None
            Department=None
            Additional_Posts=None
        
    #contact page variables
    First=" ".join(fir)
    Second=" ".join(sec)
    Title=" ".join(tit)
    Suffix=" ".join(suf)
    Website=informationdict.get("Website")
    Twitter=informationdict.get("Twitter")
    Facebook=informationdict.get("Facebook")
    
    Parliament=informationdict.get("Parliamentary")
    if Parliament!=None:
        parlphone=Parliament.get("Phone:")
        parlemail=Parliament.get("Email:")
        parladd=list(Parliament.get("Address"))
        parladd1=", ".join(parladd[:-2])
        if len(parladd)==1:
            parladd2=None
            parladd3=None
        else:
            parladd2=parladd[-2]
            parladd3=parladd[-1]
    
    Constituency=informationdict.get("Constituency")
    if Constituency!=None:
        consphone=Constituency.get("Phone:")
        consemail=Constituency.get("Email:")
        consadd=list(Constituency.get("Address"))
        consadd1=", ".join(consadd[:-2])
        
        if len(consadd)==1:
            consadd2=None
            consadd3=None
        else:
            consadd2=consadd[-2]
            consadd3=consadd[-1]
    #print(consadd2)
    #print(consadd3)
    else:
        consphone=None
        consemail=None
        consadd1=None
        consadd2=None
        consadd3=None
   
    Departmentu=informationdict.get("Departmental")
    
    if Departmentu!=None:
        depphone=Departmentu.get("Phone:")
        
        depemail=Departmentu.get("Email:")
        #depadd=Departmentu.get("Address")
        depadd=list(Departmentu.get("Address"))
        depadd1=", ".join(depadd[:-2])
        
        if len(depadd)==1:
            depadd2=None
            depadd3=None
        else:
            depadd2=depadd[-2]
            depadd3=depadd[-1]
            
        
      
    else:
        depphone=None
        depemail=None
        #depadd=None
        depadd1=None
        depadd2=None
        depadd3=None
        
    ExternalPO=informationdict.get("External/Private Office")
    if ExternalPO!=None:
        POphone=ExternalPO.get("Phone:")
        POemail=ExternalPO.get("Email:")
        #POadd=ExternalPO.get("Address")
        POadd=list(ExternalPO.get("Address"))
        POadd1=", ".join(POadd[:-2])
        
        if len(POadd)==1:
            POadd2=None
            POadd3=None
        else:
            POadd2=POadd[-2]
            POadd3=POadd[-1]
    else:
        POphone=None
        POemail=None
        #POadd=None
        POadd1=None
        POadd2=None
        POadd3=None
   
    datas=["MP", Title, First, Second, Suffix, Position, Department,\
           parlemail, parlphone, parladd1, parladd2, parladd3,\
           Committees1, Otherpost1, Additional_Posts, Party,\
           consemail, consphone, consadd1, consadd2, consadd3,\
           POemail, POphone, POadd1, POadd2, POadd3,\
           depphone, depemail, depadd1, depadd2, depadd3,\
           Website, Twitter, Facebook]
    
    MPsh.append(datas)
    
  

wb.save('Test.xlsx')


In [ ]:
#Lordcareer&Contact
start1 = 'full title is '
end1 = ', and'
start2 = 'given name is '
end2 = '.'
count=0
for Lord in activeLord:
    
    fullname3=Lord[0].find("h1").string.strip()
    #print(fullname3)
    current=Lord[0].find("div", {"card-box"})
    h4categories=current.find_all("h4")
    h4catstrips=[h4.string.strip() for h4 in h4categories]
    divcategories=current.find_all("div", {"card-list"})
    informationdict={}
    for h4,div in zip(h4catstrips,divcategories):
        if "Representation" in h4:
            info=div.find("div", {"class":"primary-info"}).string.strip()
        elif "Party affiliation" in h4:
            info=div.find("div", {"class":"primary-info"}).string.strip()
        else:
            if "Committee" in h4:
                infon=div.find_all("div", {"class":"primary-info"})
                info=[]
                for info1 in infon:
                    a=info1.string.strip()
                    info.append(a)
            elif "Other Post" in h4:
                infon=div.find_all("div", {"class":"primary-info"})
                info=[]
                for info1 in infon:
                    a=info1.string.strip()
                    
                    info.append(a)
                    
            else:
                info1=div.find("div", {"class":"primary-info"}).string.strip()
                try:
                    info2=div.find("div", {"class":"secondary-info"}).string.strip()
                except:
                    info2=None
                info=[info1,info2]
        informationdict[h4]=info
        
#Lord contact
    #fullname=Lord[1].find("h1").string.strip()
    #fullname3=Lord[0].find("h1").string.strip()
    #print(fullname3)
    descriptivesentence=str(Lord[0].find("p"))
    #print(descriptivesentence)
    fullname = re.search('%s(.*)%s' % (start1, end1), descriptivesentence).group(1)
    christianname1 = re.search('%s(.*)%s' % (start2, end2), descriptivesentence).group(1)
    #christianname=christianname1[:-1]
    #print(fullname)
    #print(christianname1)
    
    Christianfir=[]
    Christiansec=[]
    splitnamchris=[]
    splitnamchris = christianname1.split(" ")
    
    for index, namez in enumerate(splitnamchris):
        if index==0:
            Christianfir.append(namez)
        else:
            Christiansec.append(namez)
            
    fir=[]
    sec=[]
    tit=[]
    suf=[]
    namesplit=[]
    splitnam=fullname.split(" ")
    for snam in splitnam:
        if snam in titles:
            tit.append(snam)
        elif snam in suffixes:
            suf.append(snam)
        else:
            namesplit.append(snam)
    for index, namez in enumerate(namesplit):
        if index==0:
            fir.append(namez)
        else:
            sec.append(namez)
    
    contmethods=Lord[1].find_all("div",{"class":"card-inner"})
    for contmethod in contmethods:
        method=contmethod.find("div", {"class":"primary-info"}).string.strip()
        if "Parliamentary" in method:
            detail={}
            tradconts=contmethod.find_all("div", {"class":"contact-line"})
            try:
                addressall=contmethod.find("div", {"class":"col-md-5"}).get_text("\n").strip()
                addresssplit=addressall.splitlines()
            except:
                addressall=None
            
            
            for tradcont in tradconts:
                label=tradcont.find("span", {"class":"label"}).string
                inform=tradcont.find("a").string.strip()
                detail[label]=inform
            detail["Address"]=addresssplit
        
      
     ######################################################   
        elif "External/Private" in method:
            detail={}
            tradconts=contmethod.find_all("div", {"class":"contact-line"})
            addressall=contmethod.find("div", {"class":"col-md-5"}).get_text("\n").strip()
            addresssplit=addressall.splitlines()
            for tradcont in tradconts:
                label=tradcont.find("span", {"class":"label"}).string.strip()
                inform=tradcont.find("a").string.strip()
                detail[label]=inform
            detail["Address"]=addresssplit
            
        elif "Department" in method:
            detail={}
            tradconts=contmethod.find_all("div", {"class":"contact-line"})
            try:
                addressall=contmethod.find("div", {"class":"col-md-5"}).get_text("\n").strip()
                addresssplit=addressall.splitlines()
            except:
                addressall=None
            
            for tradcont in tradconts:
                label=tradcont.find("span", {"class":"label"}).string.strip()
                inform=tradcont.find("a").string.strip()
                detail[label]=inform
            detail["Address"]=addresssplit
        
        else:
            try:
                detail=contmethod.find("div", {"class":"indicator indicator-label"}).string.strip()
            except:
                detail=None
        methanddet=[method,detail]
        informationdict[method]=detail
        
    #career page variables  
    
    Party=informationdict.get("Party affiliation")
    OtherPost=informationdict.get("Other posts")
    if OtherPost!=None:
        Otherpost1=", ".join(filter(None,OtherPost))
    else:
        Otherpost1=None
    Committees=informationdict.get("Committee memberships")
    if Committees!=None:
        com_count=collections.Counter(Committees)
        chairlist=[]
        for key, value in com_count.items():
            if value > 1:
                chairlist.append(key)
        if chairlist!=None:
            for chaircom in chairlist:
                x=int(Committees.index(chaircom))
                Committees.insert(x, "CHAIR")
                Committees.remove(chaircom)
        Committees1=", ".join(Committees)
    else:
        Committees1=None
        
    posdep=[informationdict.get("Government post"),informationdict.get("Government posts"),\
informationdict.get("Opposition post"), informationdict.get("Opposition posts")]
    print(posdep)
    for postdescription in posdep:
        if postdescription==None:
            continue
        
        else:
            print(postdescription)
            thing=[]
            if type(postdescription[0])==list:
                print("lippy")
                Position=postdescription[0][0]
                Department=postdescription[0][1]
                print(Position)
                print(Department)
                postdescription.pop(0)
                for a in postdescription:
                #print(fullname3)
                    if a==None:
                        continue
                    x=", ".join(a)
                    thing.append(x)
                Additional_Posts="/".join(thing)
            else:
                Position=postdescription[0]
                Department=postdescription[1]
                print(Position)
                print(Department)
                Additional_Posts=None
    if all(postdescription is None for postdescription in posdep):
        Position=None
        Department=None
        Additional_Posts=None
        
    #contact page variables
    First=" ".join(fir)
    #print(First)
    Second=" ".join(sec)
    Title=" ".join(tit)
    Suffix=" ".join(suf)
    Christianfirst=" ".join(Christianfir)
    Christiansecond=" ".join(Christiansec)
    Website=informationdict.get("Website")
    Twitter=informationdict.get("Twitter")
    Facebook=informationdict.get("Facebook")
    
    Parliament=informationdict.get("Parliamentary")
    if Parliament!=None:
        parlphone=Parliament.get("Phone:")
        parlemail=Parliament.get("Email:")
        parladd=list(Parliament.get("Address"))
        parladd1=parladd[0]
        if len(parladd)==1:
            parladd2=None
            parladd3=None
        else:
            parladd2=parladd[-2]
            parladd3=parladd[-1]
    
   
    Departmentu=informationdict.get("Departmental")
    if Departmentu!=None:
        depphone=Departmentu.get("Phone:")
        depemail=Departmentu.get("Email:")
        depadd=list(Departmentu.get("Address"))
        depadd1=", ".join(depadd[:-2])
        
        if len(depadd)==1:
            depadd2=None
            depadd3=None
        else:
            depadd2=depadd[-2]
            depadd3=depadd[-1]
            
        
      
    else:
        depphone=None
        depemail=None
        #depadd=None
        depadd1=None
        depadd2=None
        depadd3=None
        
    ExternalPO=informationdict.get("External/Private Office")
    if ExternalPO!=None:
        POphone=ExternalPO.get("Phone:")
        POemail=ExternalPO.get("Email:")
        #POadd=ExternalPO.get("Address")
        POadd=list(ExternalPO.get("Address"))
        POadd1=", ".join(POadd[:-2])
        
        if len(POadd)==1:
            POadd2=None
            POadd3=None
        elif len(POadd)==0:
            POadd1=None
            POadd2=None
            POadd3=None
        else:
            print(POadd)
            print(First, Second)
            POadd2=POadd[-2]
            POadd3=POadd[-1]
    else:
        POphone=None
        POemail=None
        #POadd=None
        POadd1=None
        POadd2=None
        POadd3=None
   
 
    datas=["Lord", Title, First, Second, Suffix, Position, Department, \
           parlemail, parlphone, parladd1, parladd2, parladd3,\
           Christianfirst, Christiansecond,\
           Committees1, Otherpost1, Additional_Posts, Party,\
           POemail, POphone, POadd1, POadd2, POadd3,\
           depphone, depemail, depadd1, depadd2, depadd3,\
           Website, Twitter, Facebook]
    
    Lordsh.append(datas)
    count+=1
#print(count)

    

wb.save('Test.xlsx')


In [125]:
print(activeLord[10])

[<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport">
<link href="/dist/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<title>An error has occurred - MPs and Lords - UK Parliament</title>
<meta content="An error has occurred - MPs and Lords - UK Parliament" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://members.parliament.uk/Error/500" property="og:url"/>
<meta content="https://members.parliament.uk/dist/opengraph-card.png" property="og:image"/>
<meta content="summary_large_image" name="twitter:card"/>
<link href="/dist/app.css?v=HlHY1DtczWCsgx0XjkfW73P1y4Jy4AoeVVSJv0aqxT4" rel="stylesheet">
<script type="text/javascript">
	
    var appInsights=window.appInsights||function(a){
        function b(a){c[a]=function(){var b=arguments;c.queue.push(function(){c[a].apply(c,b)})}}var c={config:a},d=

In [ ]:
#####

In [10]:
#compiling list of all previous HoC member urls in list r, only use 1000 urls max
r=[]
for i in range(4538,4877):      #Number of pages is 4877 ignore 4537
    url = "https://members.parliament.uk/member/{}/career".format(i)
    page=requests.get(url).content
    soup = BeautifulSoup(page, 'html.parser')
    url1 = "https://members.parliament.uk/member/{}/contact".format(i)
    page=requests.get(url1).content
    soup1 = BeautifulSoup(page, 'html.parser')
    r.append([soup,soup1])

In [12]:
#checks whether to append to lord list or mp list
lordtitles=["Lord","Viscount","Baroness","Earl","Archbishop","Lady ", "Countess", "Marquess", "Duke", "Bishop"]
MP1=[]
Lord1=[]

for member in r:
    if any(lordtitle in member[0].h1.string.strip() for lordtitle in lordtitles):
        
        #print(Lord1)
        if " Viscount" in member[0].h1.string.strip():
            MP1.append(member)
        elif "Lady Olga" in member[0].h1.string.strip():
            continue
        elif "Jonathan Lord" in member[0].h1.string.strip():
            MP1.append(member)
        elif "Marquess of Hamilton" in member[0].h1.string.strip():
            continue
        elif member[0].find("h3")==None:
            continue
        else:
            Lord1.append(member)
    else:
        MP1.append(member)
print(len(Lord1))


94


In [13]:
#checks if an active mp and if so appends to active list
activeMP=[]
for member in MP1:
    if "does not exist" in member[1].h1.string.strip():
        print("meh")
    else:
        if member[0]!=member[1]:
            activeMP.append(member)
print(len(activeMP))


meh
meh
meh
meh
208


In [14]:
#checks if an active lord and if so appends to active list
activeLord=[]
for member in Lord1:
    #print(member[0].find("h1"))
    if "Current" in str(member[0].find("h3")) or member[0]!=member[1]:
        activeLord.append(member)
print(len(activeLord))

94
